In [16]:
import pandas as pd

df = pd.read_csv('data/시도별사교육참여율(년도).csv')
df.head(), df.shape, df['지역별'].unique()


(     시점   지역별  사교육참여율
 0  2009  대도시외    68.1
 1  2009  대도시외    77.4
 2  2009  대도시외    75.6
 3  2009  대도시외    74.0
 4  2009   광역시    75.9,
 (250, 3),
 array(['대도시외', '광역시', '서울'], dtype=object))

In [17]:
import pandas as pd

# 시점 변환
df['시점'] = pd.to_datetime(df['시점'], format='%Y', errors='coerce')

# 월별 데이터 프레임 생성
monthly_df = pd.DataFrame()

# 지역별로 그룹화 후 평균 계산
grouped = df.groupby(['시점', '지역별'])['사교육참여율'].mean().reset_index()

# 각 지역별 데이터를 추가
for region in grouped['지역별'].unique():
    # 해당 지역 데이터 추출 및 인덱스 설정
    yearly_data = grouped[grouped['지역별'] == region].set_index('시점')['사교육참여율']
    
    # 보간을 위한 리샘플링
    monthly_data = yearly_data.resample('M').mean()  # 평균으로 리샘플링
    monthly_data = monthly_data.interpolate(method='linear')  # 선형 보간
    
    # 결측값 제거
    monthly_data = monthly_data.dropna()
    
    # 지역별 데이터 추가
    monthly_df[region] = monthly_data.astype(int)

# 인덱스를 시점으로 설정
monthly_df.index.name = '시점'
monthly_df.reset_index(inplace=True)

# 시점 포맷 변환
monthly_df['시점'] = monthly_df['시점'].dt.strftime('%Y.%m')

monthly_df.head(12), monthly_df.shape

# # 데이터 저장
# monthly_df.to_csv('시도별사교육참여율(월별).csv')


C:\Users\Administrator\AppData\Local\Temp\ipykernel_272\3223346247.py:18: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data = yearly_data.resample('M').mean()  # 평균으로 리샘플링
C:\Users\Administrator\AppData\Local\Temp\ipykernel_272\3223346247.py:18: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data = yearly_data.resample('M').mean()  # 평균으로 리샘플링
C:\Users\Administrator\AppData\Local\Temp\ipykernel_272\3223346247.py:18: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data = yearly_data.resample('M').mean()  # 평균으로 리샘플링


(         시점  광역시  대도시외  서울
 0   2009.01   74    70  79
 1   2009.02   74    70  79
 2   2009.03   74    70  79
 3   2009.04   74    70  79
 4   2009.05   74    70  78
 5   2009.06   74    69  78
 6   2009.07   74    69  78
 7   2009.08   74    69  78
 8   2009.09   74    69  78
 9   2009.10   74    69  78
 10  2009.11   74    69  77
 11  2009.12   74    69  77,
 (169, 4))